In [ ]:
import os
import platform
if platform.system() == "Windows":
    try:
        import scrapy
        from scraper_api import ScraperAPIClient
        from fake_headers import Headers
    except ImportError:
        os.system('python -m pip install scrapy')
        os.system('python -m pip install scraperapi_sdk')
        os.system('python -m pip install fake_headers')
else:
    try:
        import scrapy
        from scraper_api import ScraperAPIClient
        from fake_headers import Headers
    except ImportError:
        os.system('python3 -m pip install scrapy')
        os.system('python3 -m pip install scraperapi_sdk')
        os.system('python3 -m pip install fake_headers')


import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
import csv
import os


class WinemagSpider(scrapy.Spider):
    name = 'winemag'
    allowed_domains = ['winemag.com']
    start_urls = ['https://www.winemag.com/ratings/']

    custom_settings = {
        "DOWNLOAD_DELAY":1
    }

    filepath = os.getcwd()

    if filepath+"/../data/Scraped Data/winemag.csv" not in os.listdir(os.getcwd()):
        with open(filepath+"/../data/Scraped Data/winemag.csv","a",newline="",encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow(['url','title','badge','description','author','rating','price','designation','variety','appellation','winery','alcohol',
                             'bottle_size','category','importer','date_published','user_avg_rating'])
    
    alreadyscrapped = []
    with open(filepath+"/../data/Scraped Data/winemag.csv","r") as r:
        reader = csv.reader(r)
        for line in reader:
            alreadyscrapped.append(line[0])
    
    def start_requests(self):
        yield scrapy.Request(
            url="https://www.winemag.com/wp-admin/admin-ajax.php?action=getSearchReviews&filters[drink_type]=wine&filters[page]=1&filters[sort_by]=pub_date_web&filters[sort_dir]=desc&filters[stock_no]=&filters[reviewer]=&filters[bestbuy_year]=&filters[search_type]=&filters[s]=&template=&isBestBuy=",
            callback=self.parse,
            meta={
                "url":"https://www.winemag.com/wp-admin/admin-ajax.php?action=getSearchReviews&filters[drink_type]=wine&filters[page]=1&filters[sort_by]=pub_date_web&filters[sort_dir]=desc&filters[stock_no]=&filters[reviewer]=&filters[bestbuy_year]=&filters[search_type]=&filters[s]=&template=&isBestBuy=",
                "page":2
            }
        )


    def parse(self, response,*args):
        sel = scrapy.Selector(text=response.json()['results'])
        links = sel.xpath('.//*[@class="review-listing row"]/@href').extract()
        for link in links:
            if link not in self.alreadyscrapped:
                yield scrapy.Request(link,callback=self.getdata,meta={
                    'link':link
                })
            else:
                print("Exists...")

        if links:
            yield scrapy.Request(
                url=response.meta.get('url').replace('[page]=1','[page]='+str(response.meta.get('page'))),
                callback=self.parse,
                meta={
                    'url':response.meta.get('url'),
                    'page':response.meta.get('page')+1
                }
            )


    def getdata(self,response):
        title = response.xpath('.//h1/text()').extract_first()
        badge = response.xpath('.//*[@class="badge"]/span/text()').extract_first()
        try:
            description = response.xpath('.//*[@class="description"]/text()').extract_first().strip()
        except:
            description = ''
        author = response.xpath('.//*[@class="description"]/span/a/text()').extract_first()
        rating = response.xpath('.//li[contains(.,"rating")]/div[2]/span/span/text()').extract_first()
        price = response.xpath('.//li[contains(.,"Price")]/div[2]/span/span/text()').extract_first()
        designation = response.xpath('.//li[contains(.,"Designation")]/div[2]/span/span/text()').extract_first()
        try:
            variety = ', '.join(response.xpath('.//li[contains(.,"Variety")]/div[2]/span/a/text()').extract())
        except:
            variety = ''
        try:
            appellation = ', '.join(response.xpath('.//li[contains(.,"Appellation")]/div[2]/span/a/text()').extract())
        except:
            appellation = ''
        try:
            winery = ', '.join(response.xpath('.//li[contains(.,"Winery")]/div[2]/span/span/a/text()').extract())
        except:
            winery = ''

        alcohol = response.xpath('.//li[contains(.,"Alcohol")]/div[2]/span/span/text()').extract_first()
        bottle_size = response.xpath('.//li[contains(.,"Bottle Size")]/div[2]/span/span/text()').extract_first()
        category = response.xpath('.//li[contains(.,"Category")]/div[2]/span/span/text()').extract_first()
        importer = response.xpath('.//li[contains(.,"Importer")]/div[2]/span/span/text()').extract_first()
        date_published = response.xpath('.//li[contains(.,"Date Published")]/div[2]/span/span/text()').extract_first()
        user_avg_rating = response.xpath('.//li[contains(.,"User Avg Rating")]/div[2]/span/span/span/text()').extract_first()

        with open(self.filepath+"/../data/Scraped Data/winemag.csv","a",newline="",encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerow([response.meta.get('link'),title,badge,description,author,rating,price,designation,variety,appellation,winery,alcohol,
                             bottle_size,category,importer,date_published,user_avg_rating])
            print([response.meta.get('link'),title,badge,description,author,rating,price,designation,variety,appellation,winery,alcohol,
                             bottle_size,category,importer,date_published,user_avg_rating])
            

process = CrawlerProcess(get_project_settings())
process.crawl(WinemagSpider)
process.start()

